In [3]:
import pandas as pd
import numpy as np

import re
import nltk
# nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = list(stopwords.words('english'))

import spacy
nlp = spacy.load("en_core_web_sm", disable=["tagger","parser", "ner"])


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import recall_score, classification_report

import matplotlib.pyplot as plt

## Import pre-processed data

Write a function to import pre-processed data for modelling. Currently, just reading from a previously saved csv file.


In [4]:
data = pd.read_csv("data/lemmatized_dev_data.csv")

emotions = data['label'].unique().tolist()
emotions.sort()
emotions

['anger', 'fear', 'joy', 'love', 'sadness', 'surprise']

## TF IDF Vectorization

In [21]:
# Create a DenseTransformer since TF-IDF vectorization returns sparse matrices
class DenseTransformer(TransformerMixin):
    def transform(self, X, y=None, **fit_params):
        return X.toarray()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

In [74]:
pipe = Pipeline([
    ('tfidf', TfidfVectorizer(min_df = 1e-3, max_df= 0.9, stop_words= list(stopwords.words('english')), ngram_range= (1,2))),
    ('dense', DenseTransformer()),
    ('knn', KNeighborsClassifier())
])
# params = {'knn__n_neighbors': range(5,10,5)}
# pipe_grid = GridSearchCV(pipe, param_grid= params, cv= 5)
# pipe_grid.fit(data['process_text'], data['label'])

# print(pipe_grid.best_params_)
# model = pipe_grid.best_estimator_

pipe.fit(data['process_text'], data['label'])
model = pipe

In [75]:
preds = model.predict(data['process_text'])

In [ ]:
X_test = data.head(100).copy()
X_test['pred_label'] = model.predict(X_test['process_text'])
X_test

,text,label,process_text,pred_label
0,i usually wear my hair in a twist out and when...,joy,i usually wear my hair in a twist out and when...,joy
1,i feel so irritated and rejected by it or is g...,anger,i feel so irritated and rejected by it or is g...,joy
2,i can feel it disturbed within me,sadness,i can feel it disturbed within me,joy
3,im talking about down right i feel like slappi...,anger,i m talking about down right i feel like slapp...,anger
4,i almost feel like im messing with a precious ...,joy,i almost feel like i m messing with a precious...,joy
...,...,...,...,...
95,i feel glad because i have my best friends aro...,joy,i feel glad because i have my best friends aro...,joy
96,i think it impossible for anyone who is paying...,joy,i think it impossible for anyone who is paying...,joy
97,@williamcontrol just preordered The Pale EP......,fear,williamcontrol just preordered the pale ep wou...,sadness
98,id never feel op or hesitant to do things duri...,fear,i d never feel op or hesitant to do things dur...,sadness


In [ ]:
print(classification_report(data['label'], preds, labels= emotions))

              precision    recall  f1-score   support

       anger       0.23      0.18      0.20     46295
        fear       0.24      0.13      0.17     39253
         joy       0.38      0.72      0.50    112083
        love       0.21      0.05      0.08     27146
     sadness       0.39      0.23      0.29     96388
    surprise       0.21      0.02      0.04     11768

    accuracy                           0.35    332933
   macro avg       0.28      0.22      0.21    332933
weighted avg       0.33      0.35      0.31    332933

